In [1]:
#连接ERP的DB2数据库，获取TACAIA2（前台：ACAI25）,TACACM4(ACAC80非产成品),TACACMM(ACACMM),TACACJ5(废钢入库履历)
import ibm_db
import pandas as pd
connStr='DATABASE=BHBXA1;HOSTNAME=192.168.2.28;PORT=50000;PROTOCOL=TCPIP;UID=bhbxa1;PWD=bhbxa1'
con=ibm_db.connect(connStr,'','')


In [2]:
import ibm_db_dbi

In [3]:
conn=ibm_db_dbi.Connection(con)

In [4]:
tacaia2=pd.read_sql('select * from tacaia2',conn,parse_dates={'APP_TRNC_TIME':'%Y%m%d%H%M%S'})

In [5]:
from pandas.api.types import CategoricalDtype
tacaia2['ACCOUNT_TITLE_ITEM']=tacaia2['ACCOUNT_TITLE_ITEM'].astype(CategoricalDtype(['01','33','03','31','50','51']))#这里对账务代码是01-33-03-31-50-51的顺序，因为先有产出这个材料，再有后续的专用充当和投入或销售，这里暂不考虑一个材料号被二次以上转用的情况，如上一个木材料转用经过工序加工后，转用产出一个材料号，当期这个材料又直接被专用挂了另外的订单号的情况
tacaia2['ACCOUNT_TITLE_ITEM'].cat.categories

Index(['01', '33', '03', '31', '50', '51'], dtype='object')

In [6]:
tacaia2.set_index(['ACCOUNT_TITLE_ITEM','MAT_NO','UNIT_CODE','SALE_ORDER_SUB_NO'],inplace=True) #将账务代码提到顺序第1位，以方便下面在同一投入账务代码下，按有投入的材料号迭代。

In [7]:
tacaia2.insert(tacaia2.shape[1],'MAT_Value',0.00)
tacaia2.insert(tacaia2.shape[1],'Lost_Value',0.00)

In [37]:
tacacm4.loc['00037Z0785','PERIOD_START_N']

27636.38

In [ ]:
def InputPrices(MAT_NO,PRODUCT_CODE,MTRL_NO,MAT_WT):
    if PRODUCT_CODE[0] in ['0','1','2']:
        MatPrice=(tacacm4.loc[PRODUCT_CODE,'PRST_VRNC_AMOUNT']+tacacm4.loc[PRODUCT_CODE,'OTHR_INCM_VRNC_AMT']-tacacm4.loc[PRODUCT_CODE,'OTHER_SEND_VRNC_AMT']+tacacm4.loc[PRODUCT_CODE,'CRRT_OTPT_VRNC_AMT']+tacacm4.loc[PRODUCT_CODE,'IN_MATCH_VRNC_AMT'])/(tacacm4.loc[PRODUCT_CODE,'PERIOD_START_N']+tacacm4.loc[PRODUCT_CODE,'OTHERS_RECEIVE_N']-tacacm4.loc[PRODUCT_CODE,'OTHERS_SEND_N']+tacacm4.loc[PRODUCT_CODE,'CURRENT_N']+tacacm4.loc[PRODUCT_CODE,'IN_MATCH_N'])#投入材料单价=（期初金额+其他收入-其他支出+本期产出+本期转入）的金额/重量
    elif PRODUCT_CODE[0] in ['3','4','5','9']:
        MatPrice=(tacacmm.loc[PRODUCT_CODE,'START_AMOUNT']+tacacmm.loc[PRODUCT_CODE,'OTHR_INCM_ACT_AMT']-tacacmm.loc[PRODUCT_CODE,'OTHR_SEND_ACT_AMT']+tacacmm.loc[PRODUCT_CODE,'CRRT_OTPT_ACT_AMT']+tacacmm.loc[PRODUCT_CODE,'NEXT_MATCH_AMT']
)/(tacacmm.loc[PRODUCT_CODE,'PERIOD_START_N']+tacacmm.loc[PRODUCT_CODE,'OTHERS_RECEIVE_N']-tacacmm.loc[PRODUCT_CODE,'OTHERS_SEND_N']+tacacmm.loc[PRODUCT_CODE,'CURRENT_N']+tacacmm.loc[PRODUCT_CODE,'NEXT_MATCH_USED_N']
)#投入材料单价=（期初金额+其他收入-其他支出+本期产出+本期转入）的金额/重量
    MatValue=MatPrice*MAT_WT
    
    LostPercent=0.4#次板和二级品的损失百分比（以后如果要跟着系统参数表，到时可另从数据库取数，这里暂时设成常用的40%）
    #判断投入材料号为参数中材料号&物料形态数值大于3（即为二等品或废次）的计数是否大于0，是则分无聊形态为4和5、9计算损失金额；否则损失金额为0
    if tacaia2[(tacaia2.IN_MAT_NO==MAT_NO)&(int(tacaia2.MAT_TYPE)>3)].count()>0:
        LostValue=tacaia2[(tacaia2.IN_MAT_NO==MAT_NO)&(tacaia2.MAT_TYPE in ['4','5']),'MAT_WT'].sum()*MatPrice*LostPercent+tacaia2[(tacaia2.IN_MAT_NO==MAT_NO)&(tacaia2.MAT_TYPE=='9'),'MAT_WT'].sum()*(MatPrice-tacacj5.loc['900'&MTRL_NO&'000000','CRRT_YEAR_STD_AMT'])

In [ ]:
def SalePrices(PRODUCT_CODE,MAT_WT):
    if PRODUCT_CODE[0] in ['0','1','2']:
        SalePrice=(tacacm4.loc[PRODUCT_CODE,'PRST_VRNC_AMOUNT']+tacacm4.loc[PRODUCT_CODE,'OTHR_INCM_VRNC_AMT']-tacacm4.loc[PRODUCT_CODE,'OTHER_SEND_VRNC_AMT']+tacacm4.loc[PRODUCT_CODE,'CRRT_OTPT_VRNC_AMT']+tacacm4.loc[PRODUCT_CODE,'IN_MATCH_VRNC_AMT']-tacacm4.loc[PRODUCT_CODE,'NEXT_PRSP_VRNC_AMT']-tacacm4.loc[PRODUCT_CODE,'NEXT_MATCH_VRNC_AMT'])/(tacacm4.loc[PRODUCT_CODE,'PERIOD_START_N']+tacacm4.loc[PRODUCT_CODE,'OTHERS_RECEIVE_N']-tacacm4.loc[PRODUCT_CODE,'OTHERS_SEND_N']+tacacm4.loc[PRODUCT_CODE,'CURRENT_N']+tacacm4.loc[PRODUCT_CODE,'IN_MATCH_N']-tacacm4.loc[PRODUCT_CODE,'NEXT_PRSP_USED_N']-tacacm4.loc[PRODUCT_CODE,'NEXT_MATCH_USED_N'])#销售材料单价=（期初金额+其他收入-其他支出+本期产出+本期转入-本期投入-本期转出）的金额/重量
    elif PRODUCT_CODE[0] in ['3','4','5','9']:
        SalePrice=(tacacmm.loc[PRODUCT_CODE,'START_AMOUNT']+tacacmm.loc[PRODUCT_CODE,'OTHR_INCM_ACT_AMT']-tacacmm.loc[PRODUCT_CODE,'OTHR_SEND_ACT_AMT']+tacacmm.loc[PRODUCT_CODE,'CRRT_OTPT_ACT_AMT']+tacacmm.loc[PRODUCT_CODE,'NEXT_MATCH_AMT']-tacacmm.loc[PRODUCT_CODE,'NEXT_PRSP_STD_AMT']-tacacmm.loc[PRODUCT_CODE,'NEXT_MATCH_AMT']
)/(tacacmm.loc[PRODUCT_CODE,'PERIOD_START_N']+tacacmm.loc[PRODUCT_CODE,'OTHERS_RECEIVE_N']-tacacmm.loc[PRODUCT_CODE,'OTHERS_SEND_N']+tacacmm.loc[PRODUCT_CODE,'CURRENT_N']+tacacmm.loc[PRODUCT_CODE,'NEXT_MATCH_USED_N']-tacacmm.loc[PRODUCT_CODE,'NEXT_PRSP_USED_N']-tacacmm.loc[PRODUCT_CODE,'NEXT_MATCH_USED_N']
)#销售材料单价=（期初金额+其他收入-其他支出+本期产出+本期转入-本期投入-本期转出）的金额/重量
    SaleValue=SalePrice*MAT_WT

In [ ]:
def ProducePrices(INPUT_MAT,ACC_TITLE,MAT_TYPE,MAT_WT):
    tacaia2.loc[('3'+ACC_TITLE[1],Input_MAT),'']

In [8]:
tacaia2.index.names

FrozenList(['ACCOUNT_TITLE_ITEM', 'MAT_NO', 'UNIT_CODE', 'SALE_ORDER_SUB_NO'])

In [ ]:
#tacaia2.sort_index(level=['ACCOUNT_TITLE_ITEM','MAT_NO','UNIT_CODE','SALE_ORDER_SUB_NO'],inplace=True) #跟着前面重设索引，把账务代码提到第1位，对应改变索引排序

In [8]:
tacaia2.sort_index(level='MAT_NO',inplace=True) #如果按多层索引综合排序，不行，则单按账务代码排序

In [9]:
from numpy import nan as NA
tacaia2=tacaia2.dropna() #剔除第一行的空行

In [ ]:
#tacaia2.loc['31','KEY_SEQ']

In [10]:
tacaia2.index.get_level_values('MAT_NO')[0]

'B1607078FEAEDE'

In [ ]:
#tacaia2.index.get_level_values('MAT_NO')[0]==' '  #因为排序后第一行[0]为空行，所以迭代时从第二行[1]开始（上面已删除空行，不用从第二行开始了）
for mat in tacaia2.loc['31'].index.get_level_values('MAT_NO'):
    print(tacaia2.loc[('31',mat),'KEY_SEQ'])

In [12]:
tacaia2.to_excel('tacaia2-202006.xlsx')

In [ ]:
#tacaia2.to_excel('tacaia2-202005.xlsx',sheet_name='tacaia2')
#tacaia2.loc[('B2003125AFA','LF03','33','MD2005061901'),'PRODUCT_CODE'][0][:4] #Pandas多层索引的引用
#[x[0] for x in tacaia2.index] #多层索引中一层所有元素的列表

In [11]:
acc_period=tacaia2['ACCOUNT_PERIOD'][1]
print(acc_period)

202006


In [12]:
tacacm4=pd.read_sql('select * from tacacm4 where YEAR=? and MON=?',conn,params=[acc_period[:4],acc_period[4:]])
tacacm4.set_index(['PRODUCT_CODE'],inplace=True)
tacacm4.drop(['REC_CREATOR', 'REC_CREATE_TIME', 'REC_REVISOR', 'REC_REVISE_TIME','ARCHIVE_FLAG', 'COMPANY_CODE', 'COMPANY_CNAME','COST_CENTER', 'STD_PRICE',"CURRNT_STD_AMOUNT","CRRT_OTPT_STD_AMT","OTHR_INCM_STD_AMT","SALE_STD_AMOUNT","NEXT_PRSP_STD_AMT","NEXT_MATCH_STD_AMT","OTHER_SEND_STD_AMT","PERIOD_END_STD_AMT","IN_MATCH_STD_AMT"],axis=1,inplace=True)
tacacm4[["CURRENT_N","CRRT_OTPT_VRNC_AMT","SALE_N","SALE_VRNC_AMOUNT","NEXT_PRSP_USED_N","NEXT_PRSP_VRNC_AMT","NEXT_MATCH_USED_N","NEXT_MATCH_VRNC_AMT","PERIOD_END_N","PERIOD_END_VRNC_AMT","IN_MATCH_N","IN_MATCH_VRNC_AMT"]]=[0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00]

In [16]:
#tacacmm.to_excel('tacacmm-202006.xlsx')

In [26]:
#tacacm4[['PERIOD_END_N','PERIOD_END_VRNC_AMT']]#test

,PERIOD_END_N,PERIOD_END_VRNC_AMT
PRODUCT_CODE,,
0003100002,6989.00,89829.95
0003100003,5386.00,67926.77
0003100004,78769.37,985215.95
0003100006,48716.03,578597.51
0003100719,4138.88,54112.68
...,...,...
2E033D0426,0.00,0.00
2E033D0992,0.00,0.00
2F434610464,0.00,0.00


In [23]:
#tacacm4[['PERIOD_END_N','PERIOD_END_VRNC_AMT']]=[0.00,0.00]
#tacacm4[['PERIOD_END_N','PERIOD_END_VRNC_AMT']]#test

"tacacm4[['PERIOD_END_N','PERIOD_END_VRNC_AMT']]=[0.00,0.00]\ntacacm4[['PERIOD_END_N','PERIOD_END_VRNC_AMT']]#test"

In [36]:
tacacm4.columns#test

Index(['REC_CREATOR', 'REC_CREATE_TIME', 'REC_REVISOR', 'REC_REVISE_TIME',
       'ARCHIVE_FLAG', 'COMPANY_CODE', 'COMPANY_CNAME', 'YEAR', 'MON',
       'COST_CENTER', 'STD_PRICE', 'PERIOD_START_N', 'CURRNT_STD_AMOUNT',
       'PRST_VRNC_AMOUNT', 'CURRENT_N', 'CRRT_OTPT_STD_AMT',
       'CRRT_OTPT_VRNC_AMT', 'OTHERS_RECEIVE_N', 'OTHR_INCM_STD_AMT',
       'OTHR_INCM_VRNC_AMT', 'SALE_N', 'SALE_STD_AMOUNT', 'SALE_VRNC_AMOUNT',
       'NEXT_PRSP_USED_N', 'NEXT_PRSP_STD_AMT', 'NEXT_PRSP_VRNC_AMT',
       'NEXT_MATCH_STD_AMT', 'NEXT_MATCH_USED_N', 'NEXT_MATCH_VRNC_AMT',
       'OTHERS_SEND_N', 'OTHER_SEND_STD_AMT', 'OTHER_SEND_VRNC_AMT',
       'PERIOD_END_N', 'PERIOD_END_STD_AMT', 'PERIOD_END_VRNC_AMT',
       'IN_MATCH_N', 'IN_MATCH_STD_AMT', 'IN_MATCH_VRNC_AMT'],
      dtype='object')

In [33]:
tacacmm=pd.read_sql('select * from tacacmm where YEAR=? and MON=?',conn,params=[acc_period[:4],acc_period[4:]])
tacacmm.set_index(['PRODUCT_CODE'],inplace=True)
tacacmm.drop(['REC_CREATOR', 'REC_CREATE_TIME', 'REC_REVISOR', 'REC_REVISE_TIME','ARCHIVE_FLAG', 'COMPANY_CODE', 'COMPANY_CNAME', 'STD_PRICE'],axis=1,inplace=True)
tacacmm[['CURRENT_N','CRRT_OTPT_ACT_AMT','NEXT_MATCH_AMT', 'NEXT_MATCH_USED_N', 'GM_N', 'GM_ACT_AMT', 'SY_N','SY_ACT_AMT', 'LL_N','LL_ACT_AMT','PERIOD_END_N', 'PERIOD_END_ACT_AMT','NEXT_PRSP_USED_N','NEXT_PRSP_STD_AMT', 'IN_MATCH_AMT', 'IN_MATCH_N']]=[0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00]

In [27]:
tacacj5=pd.read_sql_query('select * from tacacj5 where YEAR_MON=?',conn,params=[acc_period])
tacacj5.set_index(['PRODUCT_CODE'],inplace=True)
tacacj5.drop(['REC_CREATOR', 'REC_CREATE_TIME', 'REC_REVISOR', 'REC_REVISE_TIME','ARCHIVE_FLAG', 'COMPANY_CODE', 'COMPANY_CNAME'],axis=1,inplace=True)

In [34]:
print(tacacj5.head())
print(tacacm4.head())
print(tacacmm.head())

YEAR_MON PRODUCT_CODE_CNAME  CRRT_YEAR_STD_AMT
PRODUCT_CODE                                               
9003100187     202006                                7.5663
9001000000     202006                                3.4070
9001100000     202006                                3.4070
9002000000     202006                                4.0265
9002400000     202006                                4.0265
             REC_CREATOR REC_CREATE_TIME REC_REVISOR REC_REVISE_TIME  \
PRODUCT_CODE                                                           
0003100002        001161  20200701080348                               
0003100003        001161  20200701080348                               
0003100004        001161  20200701080348                               
0003100006        001161  20200701080348                               
0003100719        001161  20200701080348                               

             ARCHIVE_FLAG COMPANY_CODE COMPANY_CNAME  YEAR MON COST_CENTER  \
PRODUCT_CO

In [ ]:
#将ACAI25表按1、材料号升序；2、账务代码：按33 03 31 01 50 51 的顺序排列；3、销售合同号按升序排列
#tacaia2[tacaia2['MAT_WT'].rank(method='dense',ascending=False)==3]

In [ ]:
#对账务代码自定义排序：先设排序的列表；将账务代码列的数据类型改成‘category’（astype)；用set_category，设置账务代码的顺序；用sort_values使排序生效，同时，也将材料号和销售合同升序排列
acc_ttl_list=['33','03','31','01','50','51']
tacaia2['ACCOUNT_TITLE_ITEM']=tacaia2['ACCOUNT_TITLE_ITEM'].astype('category')
tacaia2['ACCOUNT_TITLE_ITEM'].cat.set_category(acc_ttl_list,inplace=True)
tacaia2.sortvalues(['MAT_NO','ACCOUNT_TITLE_ITEM','SALE_ORDER_SUB_NO'],ascending=True,inplace=True)
tacaia2